In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from itertools import compress
import os

In [2]:
path= os.getcwd()
sentences = pd.read_csv(path +"\\corpus_test.csv")

In [3]:
# so we start with a dataframe of stimuli that have high correlations between variables of interest

# First look at correlations

In [4]:
#function to display medium/high correlations between variables
def color(val):
    if val ==1:
        color= 'green'
    elif abs(val)>0.2 and abs(val) <=0.3:
        color = 'yellow'
    elif abs(val)>0.3 and abs(val) <=0.4:
        color = 'orange'
    elif abs(val) > 0.4 and abs(val) < 1:
        color= 'red'
    else : 
        color =''
    return 'background-color: %s' % color

In [5]:
# check correlation table of all variables
sentences.corr(method='pearson', min_periods=1).style.applymap(color)

,duration_s,nb_syll/s,nb_phon/s,m_freqlex/s,syll_fl/s,phon_fl/s
duration_s,1.000000,-0.136466,0.042896,-0.349901,-0.358250,-0.435270
nb_syll/s,-0.136466,1.000000,0.337535,0.127788,0.437377,-0.331097
nb_phon/s,0.042896,0.337535,1.000000,0.219513,-0.014230,0.164809
m_freqlex/s,-0.349901,0.127788,0.219513,1.000000,0.365045,0.230200
syll_fl/s,-0.358250,0.437377,-0.014230,0.365045,1.000000,0.150444
phon_fl/s,-0.435270,-0.331097,0.164809,0.230200,0.150444,1.000000


In [6]:
# then we used this brute force method to reduce correlation based on the
# estimated impact of each stimulus on the global correlation values

# Diminish correlations

In [7]:
# /!\ It's a bit time consumming
expo = [1,1,1,2,2,2,3,3] # exponents
numbers = [1933,1290,985,970,804,540,250,221] # nb of trials removed per loop so the sum of "numbers" will define how many stimuli are kept 
# for example: len(sentences)- np.sum(numbers) = 200 then at the end of all iterations only 200 stimuli will remain

sthd = 5 # variance within condition per variable
tcond = [60,80,60] # final number of trials per conditions

result2 = sentences.copy()
idx = np.arange(len(result2))

# select variables of interest from the dataframe 
variable2 = ['duration_s','nb_syll/s', 'nb_phon/s', 'm_freqlex/s','syll_fl/s', 'phon_fl/s']
variable3 =['duration_s','nb_syll/s', 'nb_phon/s', 'm_freqlex/s','syll_fl/s', 'phon_fl/s']
for val,i in enumerate(expo): 
    # initialize data
    result2 = result2.iloc[idx]    
    result3 = result2[variable2]

    # estimate impact of each stimulus on correlation values: select key variables
    aret = result3[variable3].corr().unstack().drop_duplicates()[1::]
    valeur = []
    idx = []
    fro = []
    print("gate one")
    for num,word in enumerate(result3.index):
        z=result3.drop(labels=None, axis=0, index=result3.index[num], columns=None, level=None, inplace=False, errors='raise')
        v=z[variable3].corr().unstack().drop_duplicates()[1::]
        dif = abs(aret)-abs(v)
        dif=np.sum(dif*(abs(aret))**i)    
        valeur.append(dif)
        fro.append(num)
    bons = pd.DataFrame({"valeur":np.array(valeur),'index':fro})    
    idx = np.argsort(valeur)[0:-numbers[val]] 
    print("gate 2")
    idx = idx.astype(int)
   
    # remove >|<xstd stim
    idx2 = [True] *len(idx)
    for i in variable2:
        std=np.std(result2.iloc[:][i]) 
        mean=np.mean(result2.iloc[:][i])
        for num,j in enumerate(idx):
            if result2.iloc[j][i] > mean+sthd*std or result2.iloc[j][i] < mean-sthd*std:
                idx2[num]=False
    idx = list(compress(idx, idx2))
    
    idx = np.asarray(list(set(idx)))
    print('turn: '+str(val+1)+'/'+str(len(expo))+' done')

# outcome
result3 = result3.iloc[idx]
print(result3.shape)

gate one
gate 2
turn: 1/8 done
gate one
gate 2
turn: 2/8 done
gate one
gate 2
turn: 3/8 done
gate one
gate 2
turn: 4/8 done
gate one
gate 2
turn: 5/8 done
gate one
gate 2
turn: 6/8 done
gate one
gate 2
turn: 7/8 done
gate one
gate 2
turn: 8/8 done
(200, 6)


In [8]:
# check correlation table of all variables following the procedure 
result3.corr(method='pearson', min_periods=1).style.applymap(color)

,duration_s,nb_syll/s,nb_phon/s,m_freqlex/s,syll_fl/s,phon_fl/s
duration_s,1.000000,-0.159145,0.048928,0.032804,0.106988,-0.047800
nb_syll/s,-0.159145,1.000000,0.058410,0.048617,0.074904,0.008223
nb_phon/s,0.048928,0.058410,1.000000,-0.057196,0.005304,-0.057924
m_freqlex/s,0.032804,0.048617,-0.057196,1.000000,-0.136014,0.034253
syll_fl/s,0.106988,0.074904,0.005304,-0.136014,1.000000,-0.121252
phon_fl/s,-0.047800,0.008223,-0.057924,0.034253,-0.121252,1.000000
